In [27]:
import cv2
import numpy as np
import random

# Define PSO parameters
w = 0.5
c1 = 0.8
c2 = 0.9
n_particles = 100
n_iterations = 100

# Define image path
image_path = "1.png"

# Load image
img = cv2.imread('4.jfif', 0)

# cv2.imshow("Segmented Image", img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
# Call PSO function and display segmented image



In [28]:
def objective_function(position):
    # Create binary mask based on threshold value
    threshold_value = position[0]
    _, mask = cv2.threshold(img, threshold_value, 255, cv2.THRESH_BINARY)

    # Compute percentage of foreground pixels in mask
    n_pixels = np.count_nonzero(mask)
    n_foreground_pixels = np.count_nonzero(mask == 255)
    if n_pixels > 0:
        percent_foreground_pixels = n_foreground_pixels / n_pixels
    else:
        percent_foreground_pixels = 0

    # Return negative of percentage of foreground pixels as fitness value
    return -percent_foreground_pixels

In [29]:
def pso(image):
    # Initialize particles randomly
    particles = []
    for i in range(n_particles):
        particle = [random.randint(0, 255)]
        particles.append(particle)

    # Initialize particle velocities
    velocities = []
    for i in range(n_particles):
        velocity = [0]
        velocities.append(velocity)

    # Initialize best positions and fitness values
    best_positions = particles.copy()
    best_fitness_values = [objective_function(particle) for particle in particles]

    # Initialize global best position and fitness value
    global_best_position = particles[0].copy()
    global_best_fitness_value = best_fitness_values[0]

    # Perform PSO iterations
    for i in range(n_iterations):
        # Update particle velocities and positions
        for j in range(n_particles):
            # Update velocity
            velocities[j][0] = w * velocities[j][0] + c1 * random.random() * (best_positions[j][0] - particles[j][0]) + c2 * random.random() * (global_best_position[0] - particles[j][0])

            # Update position
            particles[j][0] += velocities[j][0]

            # Clip position to range [0, 255]
            particles[j][0] = max(particles[j][0], 0)
            particles[j][0] = min(particles[j][0], 255)

        # Update best positions and fitness values
        for j in range(n_particles):
            fitness_value = objective_function(particles[j])
            if fitness_value > best_fitness_values[j]:
                best_positions[j] = particles[j].copy()
                best_fitness_values[j] = fitness_value

            if fitness_value > global_best_fitness_value:
                global_best_position = particles[j].copy()
                global_best_fitness_value = fitness_value

    # Create binary mask based on global best position
    threshold_value = global_best_position[0]
    _, mask = cv2.threshold(img, threshold_value, 255, cv2.THRESH_BINARY)

    # Apply mask to original image to create segmented image
    segmented_image = cv2.bitwise_and(img, img, mask=mask)

    return segmented_image

In [30]:
segmented_image = pso(img)
cv2.imshow("Segmented Image2", segmented_image)
cv2.waitKey(0)
cv2.destroyAllWindows()